In [1]:
#this is the fisrt thing we do importing the file kaggle.json
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 96% 162M/169M [00:01<00:00, 99.5MB/s]
100% 169M/169M [00:01<00:00, 90.1MB/s]


In [3]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df=pd.read_csv('/content/sample.csv')

In [15]:
print(df.head())

   tweet_id     author_id  inbound                      created_at  \
0    119237        105834     True  Wed Oct 11 06:55:44 +0000 2017   
1    119238  ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2    119239        105835     True  Wed Oct 11 13:00:09 +0000 2017   
3    119240  VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4    119241        105836     True  Tue Oct 10 15:17:21 +0000 2017   

                                                text response_tweet_id  \
0  @AppleSupport causing the reply to be disregar...            119236   
1  @105835 Your business means a lot to us. Pleas...               NaN   
2  @76328 I really hope you all change but I'm su...            119238   
3  @105836 LiveChat is online at the moment - htt...            119241   
4  @VirginTrains see attached error message. I've...            119243   

   in_response_to_tweet_id  
0                      NaN  
1                 119239.0  
2                      NaN  
3                 

In [19]:
import os

# Afficher le contenu du répertoire pour vérifier le nom du fichier extrait
print(os.listdir('/content/'))

# Charger les données dans un DataFrame en utilisant le bon nom de fichier
df = pd.read_csv('/content/sample.csv', usecols=['tweet_id', 'text'])

# Mise en minuscules
df['text_preprocessing'] = df['text'].str.lower()

# Afficher les premières lignes du DataFrame après nettoyage
print(df.head())


['.config', 'sample.csv', 'twcs', 'customer-support-on-twitter.zip', 'kaggle.json', 'sample_data']
   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0  @applesupport causing the reply to be disregar...  
1  @105835 your business means a lot to us. pleas...  
2  @76328 i really hope you all change but i'm su...  
3  @105836 livechat is online at the moment - htt...  
4  @virgintrains see attached error message. i've...  


In [20]:
# Suppression des ponctuations
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0  applesupport causing the reply to be disregard...  
1  105835 your business means a lot to us please ...  
2  76328 i really hope you all change but im sure...  
3  105836 livechat is online at the moment  https...  
4  virgintrains see attached error message ive tr...  


In [21]:
#supprime les mots tel que "is" ,"the" , ... etc
stop_words = set(stopwords.words('english'))
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0  applesupport causing reply disregarded tapped ...  
1  105835 business means lot us please dm name zi...  
2         76328 really hope change im sure wont dont  
3  105836 livechat online moment httpstcosy94vtu8...  
4  virgintrains see attached error message ive tr...  


In [22]:
#suppression des mots frequent
from collections import Counter

# Calculer la fréquence de chaque mot dans le text
word_freq = Counter()
for text in df['text_preprocessing']:
    word_freq.update(text.split())

# Identifier les mots fréquents à supprimer
SEUIL_FRÉQUENCE = 30
stopwords = set()
for word, freq in word_freq.items():
    if freq > SEUIL_FRÉQUENCE:  # SEUIL_FRÉQUENCE est un nombre que vous définissez
        stopwords.add(word)

# Supprimer les mots fréquents de chaque texte
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0  applesupport causing reply disregarded tapped ...  
1  105835 business means lot us please dm name zi...  
2         76328 really hope change im sure wont dont  
3  105836 livechat online moment httpstcosy94vtu8...  
4  virgintrains see attached error message ive tr...  


In [23]:
#suppression des mots rares :
from collections import Counter

# Calculer la fréquence de chaque mot dans le text
word_freq = Counter()
for text in df['text_preprocessing']:
    word_freq.update(text.split())

# Identifier les mots fréquents à supprimer
SEUIL_FRÉQUENCE_RARE = 1
mots_rares = set(word for word, freq in word_freq.items() if freq <= SEUIL_FRÉQUENCE_RARE)

# Supprimer les mots rares de chaque texte
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: ' '.join([word for word in x.split() if word not in mots_rares]))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0                                 applesupport reply  
1           means lot us please dm name code concern  
2                             hope im sure wont dont  
3  105836 moment 031 031 option 1 4 3 message cal...  
4  virgintrains see message ive tried several tim...  


In [24]:
# Stemming : running --> run
ps = PorterStemmer()
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: ' '.join([ps.stem(word) for word in word_tokenize(x)]))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0                                 applesupport repli  
1             mean lot us pleas dm name code concern  
2                             hope im sure wont dont  
3  105836 moment 031 031 option 1 4 3 messag call...  
4  virgintrain see messag ive tri sever time past...  


In [25]:
# Suppression des émojis, des émoticônes, des URL, des balises HTML
df['text_preprocessing'] = df['text_preprocessing'].apply(lambda x: re.sub(r'\W', ' ', x))
print(df.head())

   tweet_id                                               text  \
0    119237  @AppleSupport causing the reply to be disregar...   
1    119238  @105835 Your business means a lot to us. Pleas...   
2    119239  @76328 I really hope you all change but I'm su...   
3    119240  @105836 LiveChat is online at the moment - htt...   
4    119241  @VirginTrains see attached error message. I've...   

                                  text_preprocessing  
0                                 applesupport repli  
1             mean lot us pleas dm name code concern  
2                             hope im sure wont dont  
3  105836 moment 031 031 option 1 4 3 messag call...  
4  virgintrain see messag ive tri sever time past...  


In [26]:
df['text_cleaned'] = df['text_preprocessing']

In [27]:
df_resultat = df[['tweet_id', 'text', 'text_cleaned']]
print(df_resultat)

    tweet_id                                               text  \
0     119237  @AppleSupport causing the reply to be disregar...   
1     119238  @105835 Your business means a lot to us. Pleas...   
2     119239  @76328 I really hope you all change but I'm su...   
3     119240  @105836 LiveChat is online at the moment - htt...   
4     119241  @VirginTrains see attached error message. I've...   
..       ...                                                ...   
88    119330  @105860 I wish Amazon had an option of where I...   
89    119331  They reschedule my shit for tomorrow https://t...   
90    119332  @105861 Hey Sara, sorry to hear of the issues ...   
91    119333  @Tesco bit of both - finding the layout cumber...   
92    119335  @105861 If that doesn't help please DM your fu...   

                                         text_cleaned  
0                                  applesupport repli  
1              mean lot us pleas dm name code concern  
2                          